In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
    
# Set alpha for EMA. alpha is the weight given to the current value
alpha = .15 
    
def read_csv(file_path):
    return pd.read_csv(file_path)

def apply_exponential_smoothing(data):
    smoothed_data = data.copy()
    for col in data.columns:
        smoothed_values = [data[col].iloc[0]]  # Start with the first actual value
        for i in range(1, len(data)):
            current_observation = data[col].iloc[i]
            previous_smoothed = smoothed_values[-1]
            smoothed_value = alpha * current_observation + (1 - alpha) * previous_smoothed
            smoothed_values.append(smoothed_value)
        smoothed_data[col] = smoothed_values
    return smoothed_data

def forecast(data, years=20):
    # First, apply smoothing to historical data
    smoothed_historical = apply_exponential_smoothing(data)
    
    # Start forecast from the last smoothed historical value
    last_smoothed = smoothed_historical.iloc[-1]
    
    forecast = pd.DataFrame(columns=data.columns)
    
    for _ in range(years):
        new_row = last_smoothed.copy()
        for col in data.columns:
            # Generate new value based on last smoothed value
            new_value = np.random.normal(last_smoothed[col], 0.05)
            # Apply exponential smoothing
            new_row[col] = alpha * new_value + (1 - alpha) * last_smoothed[col]
        
        forecast = forecast.append(new_row, ignore_index=True)
        last_smoothed = new_row
    
    return forecast

def normalize_set(data, start_col, end_col):
    subset = data.iloc[:, start_col:end_col]
    row_sums = subset.sum(axis=1)
    normalized_subset = subset.div(row_sums, axis=0)
    data.iloc[:, start_col:end_col] = normalized_subset
    return data

def normalize_forecast(forecast):
    # Flippers set
    forecast = normalize_set(forecast, 0, 4)
    # Builders set
    forecast = normalize_set(forecast, 4, 9)
    # Prospects set
    forecast = normalize_set(forecast, 9, 14)
    # Adjacents set
    forecast = normalize_set(forecast, 14, 18)
    return forecast

def write_to_excel(forecast, file_path):
    wb = Workbook()
    ws = wb.active
    ws.title = "Forecast"

    for r in dataframe_to_rows(forecast, index=False, header=True):
        ws.append(r)

    wb.save(file_path)

def main():
    input_file = "historical_transition_probabilities.csv"
    output_file = f"forecasted_transition_probabilities_{alpha}.xlsx"

    # Read CSV
    data = read_csv(input_file)
    
    # Set a random seed for repeatability
    np.random.seed(8348)

    # Perform forecast (only for future years)
    forecasted_data = forecast(data)

    # Normalize forecast
    normalized_forecast = normalize_forecast(forecasted_data)

    # Ensure all values are positive
    normalized_forecast = normalized_forecast.clip(lower=0)

    # Renormalize after clipping to ensure sum is still 1
    normalized_forecast = normalize_forecast(normalized_forecast)

    # Write only the forecasted data to Excel
    write_to_excel(normalized_forecast, output_file)

    print(f"Forecast completed and saved to {output_file}")

if __name__ == "__main__":
    main()

Forecast completed and saved to forecasted_transition_probabilities_0.15.xlsx
